# QUERY: Technology Leaders in Europe & Technology Diversification

Platform: PATSTAT on BigQuery (TIP / local)
Author: mtc.berlin / PIZnet e.V.

### BUSINESS QUESTION:
Welche Firmen in europäischen Ländern sind am aktivsten in einem bestimmten
WIPO-Technologiefeld, und in welchen ANDEREN Technologiefeldern melden 
diese Firmen noch an?

### USE CASE:
- Universitäts-Patentverwertung: Potenzielle Lizenznehmer/Partner finden
- Technologietransfer: Firmen identifizieren, die im Zielfeld aktiv sind
- Innovations-Scouting: Unerwartete Technologiefeld-Kombinationen entdecken
- Erfindungs-Erweiterung: Angrenzende Felder aufdecken

### STAKEHOLDER: 
- Universitäre Patentverwertungsgesellschaften (PVG)
- PATLIB-Zentren / Technologietransferstellen
- IHK / Wirtschaftsförderung

### DESIGN-ENTSCHEIDUNGEN:
1. person_ctry_code statt NUTS → zuverlässigere Abdeckung
2. tls230/tls901 Technology Fields statt IPC → WIPO-Standard, 
   kein Whitespace-Problem, Gewichtung eingebaut
3. Europäische Länder als Scope → breiter als nur DACH

### WIPO TECHNOLOGY FIELDS (5 Sektoren, 35 Felder):

Electrical engineering:
  1-Electrical machinery, apparatus, energy | 2-Audio-visual technology
  3-Telecommunications | 4-Digital communication | 5-Basic communication processes
  6-Computer technology | 7-IT methods for management | 8-Semiconductors

Instruments:
  9-Optics | 10-Measurement | 11-Analysis of biological materials
  12-Control | 13-Medical technology

Chemistry:
  14-Organic fine chemistry | 15-Biotechnology | 16-Pharmaceuticals
  17-Macromolecular chemistry, polymers | 18-Food chemistry
  19-Basic materials chemistry | 20-Materials, metallurgy
  21-Surface technology, coating | 22-Micro-structural and nano-technology
  23-Chemical engineering | 24-Environmental technology

Mechanical engineering:
  25-Handling | 26-Machine tools | 27-Engines, pumps, turbines
  28-Textile and paper machines | 29-Other special machines
  30-Thermal processes and apparatus | 31-Mechanical elements
  32-Transport

Other fields:
  33-Furniture, games | 34-Other consumer goods | 35-Civil engineering

### PARAMETER:
- Ziel-Technologiefeld: techn_field = 'Medical technology'    ← anpassen!
- Länder: DE, FR, GB, DK, PL, ES, IT, NO, SE, FI, CH, AT     ← anpassen!
- Zeitraum: 2015-2024 (letzte 10 Jahre)                       ← anpassen!
- Mindestanzahl: 10 Patente im Zielfeld                       ← anpassen!
*/


# TEIL A: Top-Anmelder im Ziel-Technologiefeld (Europa)


In [1]:
from epo.tipdata.patstat import PatstatClient
import pandas as pd
import time

# Connect to PATSTAT
patstat = PatstatClient(env='PROD')

def timed_query(query):
    """Execute query and return DataFrame with timing."""
    start = time.time()
    res = patstat.sql_query(query, use_legacy_sql=False)
    print(f"Query took {time.time() - start:.2f}s ({len(res)} rows)")
    return pd.DataFrame(res)

# Run the query
df = timed_query("""
WITH target_field_patents AS (
    -- Alle Anmeldungen im Ziel-Technologiefeld
    SELECT DISTINCT
        a.appln_id,
        a.appln_filing_year,
        atf.weight AS tech_relevance
    FROM tls201_appln a
    JOIN tls230_appln_techn_field atf ON a.appln_id = atf.appln_id
    JOIN tls901_techn_field_ipc tf ON atf.techn_field_nr = tf.techn_field_nr
    WHERE tf.techn_field = 'Medical technology'           -- ← TECHNOLOGIEFELD anpassen!
      AND a.appln_filing_year BETWEEN 2015 AND 2024       -- ← ZEITRAUM anpassen!
      AND atf.weight > 0.25                               -- Mindest-Relevanz
),

european_applicants AS (
    -- Anmelder aus europäischen Zielländern
    SELECT
        tfp.appln_id,
        tfp.appln_filing_year,
        tfp.tech_relevance,
        p.person_id,
        p.doc_std_name,
        p.person_ctry_code,
        p.psn_sector
    FROM target_field_patents tfp
    JOIN tls207_pers_appln pa ON tfp.appln_id = pa.appln_id
    JOIN tls206_person p ON pa.person_id = p.person_id
    WHERE pa.applt_seq_nr > 0
      AND p.person_ctry_code IN (                         -- ← LÄNDER anpassen!
          'DE', 'FR', 'GB', 'DK', 'PL', 'ES', 
          'IT', 'NO', 'SE', 'FI', 'CH', 'AT'
      )
      AND p.doc_std_name IS NOT NULL
),

top_applicants AS (
    SELECT
        doc_std_name,
        person_ctry_code,
        psn_sector,
        COUNT(DISTINCT appln_id) AS patents_target_field,
        ROUND(AVG(tech_relevance), 3) AS avg_relevance,
        MIN(appln_filing_year) AS first_filing,
        MAX(appln_filing_year) AS last_filing,
        COUNT(DISTINCT CASE WHEN appln_filing_year >= 2020 THEN appln_id END) AS recent_5y,
        COUNT(DISTINCT CASE WHEN appln_filing_year < 2020 THEN appln_id END) AS earlier_5y
    FROM european_applicants
    GROUP BY doc_std_name, person_ctry_code, psn_sector
    HAVING COUNT(DISTINCT appln_id) >= 10                 -- ← SCHWELLWERT anpassen!
)

SELECT
    doc_std_name AS firma,
    person_ctry_code AS land,
    psn_sector AS sektor,
    patents_target_field,
    avg_relevance,
    recent_5y,
    earlier_5y,
    CASE 
        WHEN earlier_5y = 0 THEN 'NEU (nur ab 2020)'
        WHEN SAFE_DIVIDE(recent_5y, earlier_5y) > 1.5 THEN '↑ WACHSEND'
        WHEN SAFE_DIVIDE(recent_5y, earlier_5y) < 0.5 THEN '↓ RÜCKLÄUFIG'
        ELSE '→ STABIL'
    END AS trend,
    first_filing,
    last_filing
FROM top_applicants
ORDER BY patents_target_field DESC
LIMIT 50;
""")

# Display results
df.to_csv('active_companies_per_technology_a.csv', index=False)
df

Query took 0.54s (50 rows)


,firma,land,sektor,patents_target_field,avg_relevance,recent_5y,earlier_5y,trend,first_filing,last_filing
0,SIEMENS HEALTHCARE GMBH,DE,COMPANY,3604,0.686,1343,2261,→ STABIL,2015,2024
1,ALCON INC,CH,COMPANY,2859,0.889,1812,1047,↑ WACHSEND,2015,2024
2,MATZINGER DAVID,CH,COMPANY,2543,0.901,1718,825,↑ WACHSEND,2015,2024
3,CILAG GMBH INT,CH,COMPANY,1792,0.940,1700,92,↑ WACHSEND,2015,2024
4,HOFFMANN LA ROCHE,CH,COMPANY,1764,0.862,851,913,→ STABIL,2015,2024
5,FRESENIUS MEDICAL CARE DEUTSCHLAND GMBH,DE,COMPANY,1709,0.874,617,1092,→ STABIL,2015,2024
6,SANOFI SA,FR,COMPANY,1629,0.957,705,924,→ STABIL,2015,2024
7,SANOFI AVENTIS DEUTSCHLAND,DE,COMPANY,1622,0.942,257,1365,↓ RÜCKLÄUFIG,2015,2024
8,AESCULAP AG,DE,COMPANY,1604,0.913,877,727,→ STABIL,2015,2024
9,BIOTRONIK SE & CO KG,DE,COMPANY,1501,0.927,938,563,↑ WACHSEND,2015,2024


# TEIL B: Technologiefeld-Diversifikation der Top-Anmelder

Für die Top-Anmelder aus Teil A:
In welchen ANDEREN der 35 WIPO-Technologiefelder sind sie aktiv?
→ Zeigt Technologiebreite und potenzielle Kooperationsfelder
→ Nutzt die eingebaute WIPO-Gewichtung für saubere Zuordnung


In [2]:
from epo.tipdata.patstat import PatstatClient
import pandas as pd
import time

# Connect to PATSTAT
patstat = PatstatClient(env='PROD')

def timed_query(query):
    """Execute query and return DataFrame with timing."""
    start = time.time()
    res = patstat.sql_query(query, use_legacy_sql=False)
    print(f"Query took {time.time() - start:.2f}s ({len(res)} rows)")
    return pd.DataFrame(res)

# Run the query
df = timed_query("""
WITH target_field_patents AS (
    SELECT DISTINCT a.appln_id
    FROM tls201_appln a
    JOIN tls230_appln_techn_field atf ON a.appln_id = atf.appln_id
    JOIN tls901_techn_field_ipc tf ON atf.techn_field_nr = tf.techn_field_nr
    WHERE tf.techn_field = 'Medical technology'           -- ← Gleich wie oben!
      AND a.appln_filing_year BETWEEN 2015 AND 2024
      AND atf.weight > 0.25
),

-- Top-Firmen identifizieren
top_firms AS (
    SELECT
        p.doc_std_name,
        p.person_ctry_code,
        COUNT(DISTINCT tfp.appln_id) AS target_count
    FROM target_field_patents tfp
    JOIN tls207_pers_appln pa ON tfp.appln_id = pa.appln_id
    JOIN tls206_person p ON pa.person_id = p.person_id
    WHERE pa.applt_seq_nr > 0
      AND p.person_ctry_code IN (
          'DE', 'FR', 'GB', 'DK', 'PL', 'ES', 
          'IT', 'NO', 'SE', 'FI', 'CH', 'AT'
      )
      AND p.doc_std_name IS NOT NULL
    GROUP BY p.doc_std_name, p.person_ctry_code
    HAVING COUNT(DISTINCT tfp.appln_id) >= 10
),

-- ALLE Patente dieser Firmen (nicht nur Zielfeld!)
all_patents_of_top_firms AS (
    SELECT DISTINCT
        p.doc_std_name,
        p.person_ctry_code,
        a.appln_id,
        a.appln_filing_year
    FROM tls201_appln a
    JOIN tls207_pers_appln pa ON a.appln_id = pa.appln_id
    JOIN tls206_person p ON pa.person_id = p.person_id
    JOIN top_firms tf ON p.doc_std_name = tf.doc_std_name
                     AND p.person_ctry_code = tf.person_ctry_code
    WHERE pa.applt_seq_nr > 0
      AND a.appln_filing_year BETWEEN 2015 AND 2024
),

-- Technologiefelder aller Patente
tech_diversification AS (
    SELECT
        ap.doc_std_name,
        ap.person_ctry_code,
        tf.techn_sector,
        tf.techn_field,
        tf.techn_field_nr,
        COUNT(DISTINCT ap.appln_id) AS patent_count,
        ROUND(AVG(atf.weight), 3) AS avg_weight,
        MIN(ap.appln_filing_year) AS first_year,
        MAX(ap.appln_filing_year) AS last_year
    FROM all_patents_of_top_firms ap
    JOIN tls230_appln_techn_field atf ON ap.appln_id = atf.appln_id
    JOIN tls901_techn_field_ipc tf ON atf.techn_field_nr = tf.techn_field_nr
    WHERE atf.weight > 0.1                                -- Low-noise filter
    GROUP BY ap.doc_std_name, ap.person_ctry_code, 
             tf.techn_sector, tf.techn_field, tf.techn_field_nr
)

SELECT
    td.doc_std_name AS firma,
    td.person_ctry_code AS land,
    td.techn_sector AS sektor,
    td.techn_field AS technologiefeld,
    td.patent_count,
    tf2.target_count AS patente_im_zielfeld,
    CASE 
        WHEN td.techn_field = 'Medical technology' THEN '★ ZIELFELD'  -- ← anpassen!
        ELSE ''
    END AS ist_zielfeld,
    ROUND(td.patent_count * 100.0 / 
        SUM(td.patent_count) OVER (PARTITION BY td.doc_std_name), 1
    ) AS anteil_pct,
    td.avg_weight,
    td.first_year,
    td.last_year
FROM tech_diversification td
JOIN top_firms tf2 ON td.doc_std_name = tf2.doc_std_name 
                  AND td.person_ctry_code = tf2.person_ctry_code
WHERE td.patent_count >= 3                                -- Noise-Filter
ORDER BY td.doc_std_name, td.patent_count DESC;

""")

# Display results
df.to_csv('active_companies_per_technology_b.csv', index=False)
df

Query took 11.58s (18121 rows)


,firma,land,sektor,technologiefeld,patent_count,patente_im_zielfeld,ist_zielfeld,anteil_pct,avg_weight,first_year,last_year
0,11 HEALTH AND TECH LIMITED,GB,Instruments,Medical technology,12,12,★ ZIELFELD,70.6,0.912,2018,2021
1,11 HEALTH AND TECH LIMITED,GB,Instruments,Measurement,5,12,,29.4,0.183,2018,2021
2,1NHALER LTD,GB,Instruments,Medical technology,21,21,★ ZIELFELD,100.0,1.000,2018,2023
3,25SEGMENTS AG,CH,Instruments,Medical technology,17,17,★ ZIELFELD,68.0,0.768,2021,2023
4,25SEGMENTS AG,CH,Electrical engineering,Computer technology,8,17,,32.0,0.389,2021,2023
...,...,...,...,...,...,...,...,...,...,...,...
18116,ZUERCHER ADRIAN,CH,Instruments,Medical technology,13,13,★ ZIELFELD,23.2,1.000,2015,2024
18117,ZUERCHER ADRIAN,CH,Chemistry,Organic fine chemistry,7,13,,12.5,0.710,2017,2022
18118,ZUERCHER ADRIAN,CH,Electrical engineering,Computer technology,3,13,,5.4,1.000,2015,2022
18119,ZURIMED TECH AG,CH,Instruments,Medical technology,54,54,★ ZIELFELD,94.7,0.978,2016,2024


# TEIL C: Technologiefeld-Kreuzmatrix (Aggregiert)

Vogelperspektive: Welche Technologiefelder (außer Zielfeld) kommen
bei den Top-Firmen am häufigsten vor?
→ Zeigt branchentypische Technologiekombinationen
→ Identifiziert "überraschende" Felder für Verwertungserweiterung

Interpretation:
Wenn 'Medical technology'-Firmen häufig auch in 'Measurement' und 
'Optics' aktiv sind, ist das erwartbar. Aber wenn viele auch in 
'Computer technology' oder 'Biotechnology' aktiv sind, zeigt das
Konvergenz-Trends und neue Verwertungsmöglichkeiten.

In [3]:
from epo.tipdata.patstat import PatstatClient
import pandas as pd
import time

# Connect to PATSTAT
patstat = PatstatClient(env='PROD')

def timed_query(query):
    """Execute query and return DataFrame with timing."""
    start = time.time()
    res = patstat.sql_query(query, use_legacy_sql=False)
    print(f"Query took {time.time() - start:.2f}s ({len(res)} rows)")
    return pd.DataFrame(res)

# Run the query
df = timed_query("""
WITH target_field_patents AS (
    SELECT DISTINCT a.appln_id
    FROM tls201_appln a
    JOIN tls230_appln_techn_field atf ON a.appln_id = atf.appln_id
    JOIN tls901_techn_field_ipc tf ON atf.techn_field_nr = tf.techn_field_nr
    WHERE tf.techn_field = 'Medical technology'           -- ← Gleich wie oben!
      AND a.appln_filing_year BETWEEN 2015 AND 2024
      AND atf.weight > 0.25
),

top_firms AS (
    SELECT p.doc_std_name, p.person_ctry_code,
           COUNT(DISTINCT tfp.appln_id) AS target_count
    FROM target_field_patents tfp
    JOIN tls207_pers_appln pa ON tfp.appln_id = pa.appln_id
    JOIN tls206_person p ON pa.person_id = p.person_id
    WHERE pa.applt_seq_nr > 0
      AND p.person_ctry_code IN (
          'DE', 'FR', 'GB', 'DK', 'PL', 'ES', 
          'IT', 'NO', 'SE', 'FI', 'CH', 'AT'
      )
      AND p.doc_std_name IS NOT NULL
    GROUP BY p.doc_std_name, p.person_ctry_code
    HAVING COUNT(DISTINCT tfp.appln_id) >= 10
),

all_patents AS (
    SELECT DISTINCT p.doc_std_name, a.appln_id
    FROM tls201_appln a
    JOIN tls207_pers_appln pa ON a.appln_id = pa.appln_id
    JOIN tls206_person p ON pa.person_id = p.person_id
    JOIN top_firms tf ON p.doc_std_name = tf.doc_std_name
                     AND p.person_ctry_code = tf.person_ctry_code
    WHERE pa.applt_seq_nr > 0
      AND a.appln_filing_year BETWEEN 2015 AND 2024
),

field_summary AS (
    SELECT
        tf.techn_sector,
        tf.techn_field,
        COUNT(DISTINCT ap.doc_std_name) AS anzahl_firmen,
        COUNT(DISTINCT ap.appln_id) AS total_patente,
        ROUND(COUNT(DISTINCT ap.appln_id) * 1.0 / 
              NULLIF(COUNT(DISTINCT ap.doc_std_name), 0), 1) AS patente_pro_firma,
        ROUND(AVG(atf.weight), 3) AS avg_weight
    FROM all_patents ap
    JOIN tls230_appln_techn_field atf ON ap.appln_id = atf.appln_id
    JOIN tls901_techn_field_ipc tf ON atf.techn_field_nr = tf.techn_field_nr
    WHERE atf.weight > 0.1
    GROUP BY tf.techn_sector, tf.techn_field
    HAVING COUNT(DISTINCT ap.doc_std_name) >= 2
)

SELECT
    techn_sector AS sektor,
    techn_field AS technologiefeld,
    CASE 
        WHEN techn_field = 'Medical technology' THEN '★ ZIELFELD'
        ELSE ''
    END AS typ,
    anzahl_firmen,
    total_patente,
    patente_pro_firma,
    avg_weight,
    -- Verwertungspotenzial-Bewertung
    CASE 
        WHEN anzahl_firmen >= 10 AND patente_pro_firma >= 20 
            THEN '●●● Breites Interesse, hohe Aktivität'
        WHEN anzahl_firmen >= 5 AND patente_pro_firma >= 10 
            THEN '●● Gutes Verwertungspotenzial'
        WHEN anzahl_firmen >= 3 
            THEN '● Nische – gezielte Ansprache'
        ELSE ''
    END AS verwertungs_signal,
    -- Überraschungsfaktor: Felder aus anderen Sektoren
    CASE 
        WHEN techn_sector != 'Instruments'                -- ← Sektor des Zielfeldes
            AND anzahl_firmen >= 5
            THEN '⚡ CROSS-SECTOR – potenzielle Erweiterung!'
        ELSE ''
    END AS cross_sector_signal
FROM field_summary
ORDER BY anzahl_firmen DESC, total_patente DESC
LIMIT 35;

""")

# Display results
df.to_csv('active_companies_per_technology_c.csv', index=False)
df

Query took 0.49s (35 rows)


,sektor,technologiefeld,typ,anzahl_firmen,total_patente,patente_pro_firma,avg_weight,verwertungs_signal,cross_sector_signal
0,Instruments,Medical technology,★ ZIELFELD,2863,151244,52.8,0.803,"●●● Breites Interesse, hohe Aktivität",
1,Instruments,Measurement,,1208,89149,73.8,0.649,"●●● Breites Interesse, hohe Aktivität",
2,Electrical engineering,Computer technology,,1158,90555,78.2,0.690,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!
3,Mechanical engineering,Other special machines,,1001,41835,41.8,0.572,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!
4,Chemistry,Chemical engineering,,954,40332,42.3,0.561,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!
5,Chemistry,Pharmaceuticals,,937,83323,88.9,0.672,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!
6,Electrical engineering,"Electrical machinery, apparatus, energy",,862,100523,116.6,0.718,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!
7,Mechanical engineering,Handling,,856,38178,44.6,0.715,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!
8,Instruments,Control,,750,34826,46.4,0.536,"●●● Breites Interesse, hohe Aktivität",
9,Chemistry,Biotechnology,,729,52724,72.3,0.622,"●●● Breites Interesse, hohe Aktivität",⚡ CROSS-SECTOR – potenzielle Erweiterung!


# TEIL D: Ländervergleich – Wer dominiert das Zielfeld?

Bonus: Zeigt die Verteilung der Top-Anmelder nach Land.
Nützlich für Stakeholder-Gespräche mit Politik/Wirtschaftsförderung.

In [4]:
from epo.tipdata.patstat import PatstatClient
import pandas as pd
import time

# Connect to PATSTAT
patstat = PatstatClient(env='PROD')

def timed_query(query):
    """Execute query and return DataFrame with timing."""
    start = time.time()
    res = patstat.sql_query(query, use_legacy_sql=False)
    print(f"Query took {time.time() - start:.2f}s ({len(res)} rows)")
    return pd.DataFrame(res)

# Run the query
df = timed_query("""
WITH target_field_patents AS (
    SELECT DISTINCT a.appln_id, a.appln_filing_year
    FROM tls201_appln a
    JOIN tls230_appln_techn_field atf ON a.appln_id = atf.appln_id
    JOIN tls901_techn_field_ipc tf ON atf.techn_field_nr = tf.techn_field_nr
    WHERE tf.techn_field = 'Medical technology'           -- ← Gleich wie oben!
      AND a.appln_filing_year BETWEEN 2015 AND 2024
      AND atf.weight > 0.25
)

SELECT
    p.person_ctry_code AS land,
    c.st3_name AS land_name,
    COUNT(DISTINCT tfp.appln_id) AS patente,
    COUNT(DISTINCT p.doc_std_name) AS anmelder,
    ROUND(COUNT(DISTINCT tfp.appln_id) * 1.0 / 
          NULLIF(COUNT(DISTINCT p.doc_std_name), 0), 1) AS patente_pro_anmelder,
    COUNT(DISTINCT CASE WHEN tfp.appln_filing_year >= 2020 THEN tfp.appln_id END) AS ab_2020,
    COUNT(DISTINCT CASE WHEN tfp.appln_filing_year < 2020 THEN tfp.appln_id END) AS vor_2020,
    CASE 
        WHEN COUNT(DISTINCT CASE WHEN tfp.appln_filing_year < 2020 THEN tfp.appln_id END) = 0 THEN NULL
        ELSE ROUND(
            COUNT(DISTINCT CASE WHEN tfp.appln_filing_year >= 2020 THEN tfp.appln_id END) * 1.0 /
            NULLIF(COUNT(DISTINCT CASE WHEN tfp.appln_filing_year < 2020 THEN tfp.appln_id END), 0), 2)
    END AS wachstumsfaktor
FROM target_field_patents tfp
JOIN tls207_pers_appln pa ON tfp.appln_id = pa.appln_id
JOIN tls206_person p ON pa.person_id = p.person_id
JOIN tls801_country c ON p.person_ctry_code = c.ctry_code
WHERE pa.applt_seq_nr > 0
  AND p.person_ctry_code IN (
      'DE', 'FR', 'GB', 'DK', 'PL', 'ES', 
      'IT', 'NO', 'SE', 'FI', 'CH', 'AT'
  )
GROUP BY p.person_ctry_code, c.st3_name
ORDER BY patente DESC;

""")

# Display results
df.to_csv('active_companies_per_technology_d.csv', index=False)
df

Query took 0.39s (12 rows)


,land,land_name,patente,anmelder,patente_pro_anmelder,ab_2020,vor_2020,wachstumsfaktor
0,DE,Germany,61787,9998,6.2,26675,35112,0.76
1,CH,Switzerland,34735,2394,14.5,18006,16729,1.08
2,FR,France,25758,4850,5.3,11740,14018,0.84
3,GB,United Kingdom,25054,4717,5.3,11198,13856,0.81
4,IT,Italy,13251,4749,2.8,6444,6807,0.95
5,SE,Sweden,8698,1222,7.1,3641,5057,0.72
6,ES,Spain,8036,4102,2.0,3836,4200,0.91
7,DK,Denmark,6713,760,8.8,2830,3883,0.73
8,AT,Austria,3774,967,3.9,1505,2269,0.66
9,PL,Poland,3665,1757,2.1,1855,1810,1.02


#### TECHNOLOGIEFELD ÄNDERN (techn_field = '...'):

* 'Medical technology'                      → MedTech
* 'Biotechnology'                           → Biotech
* 'Pharmaceuticals'                         → Pharma
* 'Computer technology'                     → IT/Software
* 'Digital communication'                   → Telko/5G
* 'Electrical machinery, apparatus, energy' → Energie
* 'Environmental technology'                → Umwelttechnik
* 'Transport'                               → Automobil/Mobilität
* 'Machine tools'                           → Maschinenbau
* 'Semiconductors'                          → Halbleiter
* 'Organic fine chemistry'                  → Feinchemie
* 'Optics'                                  → Optik
* 'Measurement'                             → Messtechnik

#### Vollständige Liste
SELECT DISTINCT techn_field FROM tls901_techn_field_ipc

#### LÄNDER ÄNDERN (person_ctry_code IN (...)):
Aktuelle Auswahl: DE, FR, GB, DK, PL, ES, IT, NO, SE, FI, CH, AT
  
* Nur DACH:     ('DE', 'AT', 'CH')
* Nur Nordics:  ('DK', 'NO', 'SE', 'FI')
* EU-27+:       Alle EU-Mitglieder via tls801_country WHERE eu_member = 'Y'
* EPO-Staaten:  via tls801_country WHERE epo_member = 'Y'

#### SEKTOR-FILTER (optional in WHERE ergänzen):
* Nur Unternehmen:    AND p.psn_sector = 'COMPANY'
* Nur Forschung/Uni:  AND p.psn_sector IN ('GOV NON-PROFIT', 'UNIVERSITY')
* Nur Firmen + Uni:   AND p.psn_sector IN ('COMPANY', 'UNIVERSITY', 'GOV NON-PROFIT')
  
#### WEIGHT-SCHWELLE:
* ' > 0.5  = starke Zuordnung (Patent ist primär in diesem Feld)'
* ' >0.25 = mittlere Zuordnung (Standard, empfohlen)'
* ' >0.1  = schwache Zuordnung (inkl. Randgebiete)'

#### ZEITRAUM:
* BETWEEN 2015 AND 2024 → letzte 10 Jahre
* BETWEEN 2020 AND 2024 → letzte 5 Jahre
* BETWEEN 2010 AND 2024 → letzte 15 Jahre
